In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
input_file = "balanced_dataset_multiclass.csv" 
output_file = "remapping_dataset_multiclass.csv"
# Đọc bằng Dask cho file lớn
ddf = dd.read_csv(input_file)

# Remap labels
ddf['label'] = ddf['label'].map({0:0, 4:1, 5:2, 6:3}, meta=('label', 'int64'))

# Shuffle (Dask không hỗ trợ shuffle trực tiếp, cần workaround)
ddf = ddf.sample(frac=1, random_state=42)

# Lưu kết quả
with ProgressBar():
    ddf.to_csv(output_file, single_file=True, index=False) 

[####################################    ] | 91% Completed | 128.48 s

In [ ]:
"""
Kiểm tra dataset sau khi xử lý
"""
df = dd.read_csv(output_file)
    
# Tính tổng số mẫu
total_samples = df.shape[0].compute()
print(f"Tổng số mẫu: {total_samples}")
    
# Tính phân bố label
label_counts = df['label'].value_counts().compute()
print("Phân bố label:")
print(label_counts.sort_index())
    
# Kiểm tra số lượng label liên tiếp (chỉ kiểm tra 1 triệu mẫu đầu để tiết kiệm tài nguyên)
df_small = df.head(1000000)
consecutive_counts = (df_small['label'].diff() == 0).sum()
print(f"Số lượng label liên tiếp trong 1 triệu mẫu đầu: {consecutive_counts}")